In [ ]:
param = {}
param['objective'] = 'multi:softprob'
param['eval_metric'] = 'mlogloss'
param['max_depth'] = 6
# param['booster'] = 'dart'
#param['booster'] = 'gblinear'
param['eta'] = 0.1
#param['subsample'] = 1.0
# param["scale_pos_weight"] = 0.5
#param['colsample_bytree'] = 0.7
param['nthread'] = 8
param['num_class'] = 3
#param['alpha'] = 0.00
#param['lambda_bias'] = 0.1
# param['lambda'] = 0
#param['min_child_weight'] = 100

# if param['booster'] == 'gblinear': param['eta'] *= 1e-4
numround = 500
# scores, , []

skf = StratifiedKFold(5, shuffle=True, random_state=0)
sc,sc_mean = [],[]
pred_train = np.zeros((len(y_train), 3))

xgbs = []

for itr, ite in skf.split(X_train, y_train.interest_level.values):

    ypred = []
    for i in range(1):
        param['seed'] = i+1

        #xgboost
        Xdatatrain = xgb.DMatrix(data=X_train.ix[itr, :].values,
                                     label=y_train.ix[itr].values)
        Xdataval = xgb.DMatrix(data=X_train.ix[ite, :].values,
                                     label=y_train.ix[ite].values)

        plst = list(param.items())
        watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]
        bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=20)

        ypred.append(bst.predict(Xdataval))        
        xgbs.append(bst)
        
    ypred = sum(ypred) / len(ypred)
    pred_train[ite] = ypred
    
    sc.append(log_loss(y_train.ix[ite, :], ypred))
#     break

print('XGB: {:.3f} +- {:.3f}'.format(np.mean(sc), np.std(sc)))

In [ ]:
skf = StratifiedKFold(5, shuffle=True, random_state=0)
folds = []
for itr, ite in skf.split(X_train, y_train.interest_level.values):
    folds += [[itr, ite]]
    skf_f = StratifiedKFold(5, shuffle=True, random_state=0)
    '''
    for itr_f, ite_f in skf_f.split(X_train.ix[itr, :], y_train.ix[itr, 'interest_level'].values):
        grouped = pd.DataFrame(X_train.ix[itr_f, :][X_train.interest_level==1].groupby(['manager_id']).count().ix[:, 0])
        grouped['manager_id'] = grouped.index
        grouped.columns = ['hi_lev', 'manager_id']
        print(len(grouped))
        tmp = pd.merge(pd.DataFrame(X_train.ix[ite_f, 'manager_id']), grouped, on='manager_id', how='left')
        #X_train.ix[ite_f, 'hi_lev'] = tmp.ix[ite_f, 'hi_lev']
        print(tmp.ix[ite_f, 'hi_lev'].isnull().sum())
    '''
    grouped = pd.DataFrame(X_train.ix[itr, :][X_train.interest_level==0].groupby(['manager_id']).count().ix[:, 0])
    grouped['manager_id'] = grouped.index
    grouped.columns = ['hi_lev', 'manager_id']
    tmp = pd.merge(pd.DataFrame(X_train.ix[ite, 'manager_id']), grouped, on='manager_id', how='left')
    X_train.ix[ite, 'hi_lev'] = tmp.ix[ite, 'hi_lev']
    
    grouped = pd.DataFrame(X_train.ix[itr, :][X_train.interest_level==2].groupby(['manager_id']).count().ix[:, 0])
    grouped['manager_id'] = grouped.index
    grouped.columns = ['mid_lev', 'manager_id']
    tmp = pd.merge(pd.DataFrame(X_train.ix[ite, 'manager_id']), grouped, on='manager_id', how='left')
    X_train.ix[ite, 'mid_lev'] = tmp.ix[ite, 'mid_lev']
    
    grouped = pd.DataFrame(X_train.ix[itr, :][X_train.interest_level==1].groupby(['manager_id']).count().ix[:, 0])
    grouped['manager_id'] = grouped.index
    grouped.columns = ['low_lev', 'manager_id']
    tmp = pd.merge(pd.DataFrame(X_train.ix[ite, 'manager_id']), grouped, on='manager_id', how='left')
    X_train.ix[ite, 'low_lev'] = tmp.ix[ite, 'low_lev']

        #for i in ite_f:
        #X_train.ix[ite_f] =  0#grouped[ X_train.ix[ite_f, 'manager_id'] ]